### Reading Raster Data in OmniSci

Example script importing a directory of raster data layers into OmniSci point tables with the same base names.

In [59]:
import glob
import gdal

In [60]:
con = omnisci_connect()

Point this somewhere Jupyter can write and your OmniSci instance has permission to read

In [61]:
local_storage = "/freenas-theExpanse/mflaxman/"

For all geotiffs in the given directory...

In [62]:
file_spec = f"{local_storage}/*.tif"

If your rasters are not in geo-coordinates, you can either use gdal to warp them, or add SRID to OmniSci CSV import.

In [63]:
reprojected_grids = []
for layer in glob.glob(file_spec):
    if 'geo' in layer:
        continue # already projected
    geotiff = layer.replace(".tif","_geo.tif")
    if not os.path.exists(geotiff):
        print(f"Reprojecting to {geotiff}")
        ds = gdal.Warp(geotiff, layer, dstSRS='EPSG:4326' )
        ds = None
    reprojected_grids.append(geotiff)
print(f"Done back-projecting {len(reprojected_grids)} grid(s) to geo")

Done back-projecting 1 grid(s) to geo


In [64]:
def convert2xyz(geotiff):
    outcsv = geotiff.replace(".tif",".csv")
    options = "-of XYZ -co ADD_HEADER_LINE=YES -co COLUMN_SEPARATOR=, "
    options += "--config GDAL_CACHEMAX 4096 " # -co NUM_THREADS=ALL_CPUS
    command = "gdal_translate " + options + geotiff + " " + outcsv
    # run from commandline - could also run from python library as gdal.Translate()
    !{command}
    return(outcsv)

build appropriate schema for raster layer value and type
note that we are merging X,Y columns into a single OmniSci Point
 
You could alternatively create a small rectangle for each cell
based on its geographic footprint.  This renders better up close, but is 4x heavier

In [65]:
def ddl(table_name):
    ddl = "omnisci_geo GEOMETRY(POINT,4326), "
    ddl += "cell_value SMALLINT"  # your type may vary
    return(ddl)

In [66]:
def etl(geotiff):
    outcsv = convert2xyz(geotiff)
    df = pd.read_csv(outcsv)
    # optional: clean up, remove no-data values, etc. here if needed
    df.to_csv(outcsv, index=None) # overwrite prior
        
    # presuming file basename is sql-legal table name:
    table_name = geotiff.split('/')[-1].replace(".tif","")
    if not table_name in con.list_tables():
        c = f"CREATE TABLE {table_name} ({ddl(table_name)})"
        try:
            con.con.execute(c)
            print(f"Table created: {table_name}")
        except:
            print(f"Failed to create table: {table_name}")

        print("Loading from csv")
        q = f"COPY {table_name} FROM '{outcsv}'"
        try:
            con.con.execute(q)
            print(f"Loaded {table_name}")
        except:
            print("Csv reading is failing")
    else:
        print(f"Warning: table already exists in OmniSci, drop first to replace {table_name}")

In [67]:
for layer in reprojected_grids:
    print(f"Converting {layer} to OmniSci table")
    outcsv = convert2xyz(layer)
    etl(layer)

Converting /freenas-theExpanse/mflaxman/la_terrain_within_5000m_of_tower_20701699_geo.tif to OmniSci table
Input file size is 279, 231
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 279, 231
0...10...20...30...40...50...60...70...80...90...100 - done.
